Дано:
файлы с данными по отработанным часам по сотрудникам подрядчиков в разрезе дат за разные месяцы для разных подразделений (одно подразделение - один файл).
 
Необходимо:
Подготовить дашборд, отображающий:
1. Общее кол-во сотрудников из выбранного подразделения, за выбранный период по всем подрядчикам. Динамика по дням.  
2. Соотношение долей работающих сотрудников (в процентах) разных подразделений за выбранный период.
3. Общее количество сотрудников, которые работали в выбранный день.
P.S. для решения задачи нужно использовать одно подключение.
 
В качестве результата необходимо представить:
- ссылку для просмотра Дашборда
- скрипты на Python, если необходимы для решения задачи
- исполняемый файл, если необходим


In [133]:
from os import walk
import pandas as pd
import datetime
from datetime import date
DATAPATH = "data/"
DAYCOLUMNSHIFT = 2 # for day 1 (1 + 2)
DEFAULTUNIT = "Подразделение1.xlsx"
SHEETNAMEPATTERN = "Часы %m-%y"

In [86]:
# getting existing units (files)

files = []
for (dirpath, dirnames, filenames) in walk(DATAPATH):
    files.extend(filenames)
    break

# getting list of dates and save to dict
files_dict = {}
for file in files:
    xl = pd.ExcelFile(DATAPATH+file)
    list_of_dates = [datetime.datetime.strptime(sheet_name, SHEETNAMEPATTERN).date() for sheet_name in xl.sheet_names]
    files_dict[file.split('.')[0]] = [file, list_of_dates]
files_dict

{'Подразделение1': ['Подразделение1.xlsx',
  [datetime.date(2023, 1, 1), datetime.date(2023, 2, 1)]],
 'Подразделение2': ['Подразделение2.xlsx',
  [datetime.date(2023, 1, 1), datetime.date(2023, 2, 1)]]}

In [129]:
from datetime import date
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
from plotly.graph_objects import Figure
import plotly.express as px


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# app = Dash(__name__)
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    html.H1(children='Task 1'),
    html.Div(children='''
        Общее кол-во сотрудников из выбранного подразделения, за выбранный период по всем подрядчикам. Динамика по дням.  
    ''', style={"font-size":"15px"}),
    
    html.Div([
        dcc.Dropdown(
                    list(files_dict.keys()),
                    id='unit-name-file',),
        dcc.DatePickerRange(
            id='my-date-picker-range',
            min_date_allowed=date(1995, 8, 5),
            max_date_allowed=datetime.datetime.now().date(),
            initial_visible_month=date(2023, 1, 1),
            end_date=datetime.datetime.now().date(),
            style={'display': 'inline-block', 'marginTop': '10px'}
        ),
        html.Div(id='output-container-date-picker-range'),
    ],style={'width': '20%', 'display': 'inline-block', 'marginTop': '20px'}),
    
    html.Div([
        dcc.Graph(
            id='my-output-plot',
        ),
        html.Div(id='my-plot-status'),
    ],style={'width': '79%', 'display': 'inline-block', 'float': 'right'})
])


@app.callback(
    Output('output-container-date-picker-range', 'children'),
    Input('my-date-picker-range', 'start_date'),
    Input('my-date-picker-range', 'end_date'))
def update_output(start_date, end_date):
    string_prefix = 'You have selected: '
    if start_date is not None:
        start_date_object = date.fromisoformat(start_date)
        start_date_string = start_date_object.strftime('%B %d, %Y')
        string_prefix = string_prefix + 'Start Date: ' + start_date_string + ' | '
    if end_date is not None:
        end_date_object = date.fromisoformat(end_date)
        end_date_string = end_date_object.strftime('%B %d, %Y')
        string_prefix = string_prefix + 'End Date: ' + end_date_string
    if len(string_prefix) == len('You have selected: '):
        return 'Select a date to see it displayed here'
    else:
        return string_prefix
    
@app.callback(
    Output('my-output-plot', 'figure'),
    Output('my-plot-status', 'children'),
    Input('my-date-picker-range', 'start_date'),
    Input('my-date-picker-range', 'end_date'),
    Input('unit-name-file', 'value'))
def show_bar(start_date, end_date, name):
    status = ""
    fig = px.bar()
    sd = date.fromisoformat(start_date)
    ed = date.fromisoformat(end_date)
    if name is not None:
        if start_date is not None:
            # open excel file
            path = files_dict[name][0]
            xl = pd.ExcelFile(DATAPATH+path)
            status += f'Unit with path {path} file opened\n'
            # gather data from excel
            data = gather_data_from_excel(xl, sd, ed)
            if len(data)>0:
                status += f'Gathered {len(data)} days\n'
            # plot and return
            return (plot_bar(data, sd, ed), status)
        else:
            return(fig, "Choose start date")
    else:
        return (fig, "Choose unit")

def gather_data_from_excel(xl: pd.ExcelFile, start_date: date, end_date: date) -> list:
    list_of_sheets_date = [datetime.datetime.strptime(sheet_name, SHEETNAMEPATTERN).date() for sheet_name in xl.sheet_names]
    # find valid month (sheets)
    for index, sheets_date in enumerate(list_of_sheets_date):
        pass
    return []

def plot_bar(data: list, start_date: date, end_date: date) -> Figure:
    return px.bar()


if __name__ == '__main__':
    app.run_server(mode="external")


Dash app running on http://127.0.0.1:8050/


In [142]:
list_of_sheets_date = [datetime.datetime.strptime(sheet_name, SHEETNAMEPATTERN).date() for sheet_name in xl.sheet_names]
# find valid month (sheets)
sd = date.fromisoformat(str(date(2023,1,23)))
ed = date.fromisoformat(str(date(2023,3,23)))
valid_sheets
# zip
for index, sheets_date in enumerate(list_of_sheets_date):
# save valid column names to list
 if (sd.year == sheets_date.year and sd.month <= sheets_date.month)
    or sd.year < 
    

2023 2023 [datetime.date(2023, 1, 1), datetime.date(2023, 2, 1)]


In [258]:
xls1 = pd.ExcelFile("data/Подразделение1.xlsx")
xls2 = pd.ExcelFile("data/Подразделение2.xlsx")
df1 = pd.read_excel("data/Подразделение1.xlsx", 0)
df2 = pd.read_excel("data/Подразделение1.xlsx", 1)
df3 = pd.read_excel("data/Подразделение2.xlsx", 0)
df4 = pd.read_excel("data/Подразделение2.xlsx", 1)

In [259]:
df1.to_csv("preprocessed_data/unit1_0.csv")
df2.to_csv("preprocessed_data/unit1_1.csv")
df3.to_csv("preprocessed_data/unit2_0.csv")
df4.to_csv("preprocessed_data/unit2_1.csv")

In [148]:
df1

,Компания,Фио,Подразделение,2023-01-01 00:00:00,2023-01-02 00:00:00,2023-01-03 00:00:00,2023-01-04 00:00:00,2023-01-05 00:00:00,2023-01-06 00:00:00,2023-01-07 00:00:00,...,2023-01-22 00:00:00,2023-01-23 00:00:00,2023-01-24 00:00:00,2023-01-25 00:00:00,2023-01-26 00:00:00,2023-01-27 00:00:00,2023-01-28 00:00:00,2023-01-29 00:00:00,2023-01-30 00:00:00,2023-01-31 00:00:00
0,Подрядчик1,1,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Подрядчик1,2,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9.4,NaN,11.8,NaN,NaN,NaN
2,Подрядчик1,3,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,11.7,NaN,11.7,NaN,NaN
3,Подрядчик1,4,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Подрядчик1,5,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,Подрядчик4 ПРР,359,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,Подрядчик4 ПРР,360,Подразделение1,NaN,11.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
360,Подрядчик4 ПРР,361,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
361,Подрядчик4 ПРР,362,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [276]:
files = []
for (dirpath, dirnames, filenames) in walk(DATAPATH):
    files.extend(filenames)
    break

# first task data
files_dict = {}
df_all_data = None
for file in files:
    xl = pd.ExcelFile(DATAPATH+file)
    df_file = pd.DataFrame()
    for sheet_name in xl.sheet_names:
        df = xl.parse(sheet_name)
        dfc = (df
        .groupby("Подразделение")
        .count()
        .drop(["Компания", "Фио"], axis=1)
        .T
        .reset_index()
        .rename(columns={"index":"date"})
        )
        df_file = pd.concat([df_file, dfc])
    df_file.reset_index(inplace=True, drop=True)
    unit_name = df_file.columns[-1]
    df_file['name'] = pd.Series([unit_name] * len(df_file), index=df_file.index)
    df_file = df_file.rename(columns={unit_name: 'workers'})
    df_file.to_csv("preprocessed_data/" + str(unit_name) + ".csv")
    if df_all_data is None:
        df_all_data = df_file.copy(deep=True)
    else:
        df_all_data = pd.concat([df_all_data, df_file])
        df_all_data.reset_index(inplace=True, drop=True)
        
df_all_data.to_csv("preprocessed_data/all.csv")
        
    # if df_all_data is None:
    #     df_all_data = df_file.copy(deep=True)
    # else:
    #     df_all_data = df_all_data.join(df_file.set_index('date'), on="date", how="outer")
        # df_all_data.reset_index(inplace=True, drop=True)

# df_all_data.to_csv("preprocessed_data/task1.csv")

In [274]:
df_file


Подразделение,date,Подразделение2,name
0,2023-01-01,47,Подразделение2
1,2023-01-02,85,Подразделение2
2,2023-01-03,47,Подразделение2
3,2023-01-04,46,Подразделение2
4,2023-01-05,60,Подразделение2
5,2023-01-06,56,Подразделение2
6,2023-01-07,47,Подразделение2
7,2023-01-08,40,Подразделение2
8,2023-01-09,61,Подразделение2
9,2023-01-10,70,Подразделение2


In [243]:
xl = pd.ExcelFile("data/Подразделение2.xlsx")
df_file = pd.DataFrame()
for sheet_name in xl.sheet_names:
    df = xl.parse(sheet_name)
    dfc = (df
    .groupby("Подразделение", )
    .count()
    .drop(["Компания", "Фио"], axis=1)
    .T
    .reset_index()
    .rename(columns={"index":"date"})
    )
    df_file = pd.concat([df_file, dfc])
df_file.reset_index(inplace=True, drop=True)

In [267]:
df1

,Компания,Фио,Подразделение,2023-01-01 00:00:00,2023-01-02 00:00:00,2023-01-03 00:00:00,2023-01-04 00:00:00,2023-01-05 00:00:00,2023-01-06 00:00:00,2023-01-07 00:00:00,...,2023-01-22 00:00:00,2023-01-23 00:00:00,2023-01-24 00:00:00,2023-01-25 00:00:00,2023-01-26 00:00:00,2023-01-27 00:00:00,2023-01-28 00:00:00,2023-01-29 00:00:00,2023-01-30 00:00:00,2023-01-31 00:00:00
0,Подрядчик1,1,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Подрядчик1,2,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9.4,NaN,11.8,NaN,NaN,NaN
2,Подрядчик1,3,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,11.7,NaN,11.7,NaN,NaN
3,Подрядчик1,4,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Подрядчик1,5,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,Подрядчик4 ПРР,359,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,Подрядчик4 ПРР,360,Подразделение1,NaN,11.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
360,Подрядчик4 ПРР,361,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
361,Подрядчик4 ПРР,362,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [268]:
(df1
.groupby("Подразделение", group_keys=True)
.count()
.drop(["Компания", "Фио"], axis=1)
.append()
)

,2023-01-01,2023-01-02,2023-01-03,2023-01-04,2023-01-05,2023-01-06,2023-01-07,2023-01-08,2023-01-09,2023-01-10,...,2023-01-22,2023-01-23,2023-01-24,2023-01-25,2023-01-26,2023-01-27,2023-01-28,2023-01-29,2023-01-30,2023-01-31
Подразделение,,,,,,,,,,,,,,,,,,,,,
Подразделение1,42,84,78,78,61,68,63,61,67,70,...,64,67,84,83,83,88,81,80,80,99


In [247]:
df_all_data.join(df_file.set_index('date'), on="date", how="outer")

Подразделение,date,Подразделение1,Подразделение2
0,2023-01-01,42,47
1,2023-01-02,84,85
2,2023-01-03,78,47
3,2023-01-04,78,46
4,2023-01-05,61,60
5,2023-01-06,68,56
6,2023-01-07,63,47
7,2023-01-08,61,40
8,2023-01-09,67,61
9,2023-01-10,70,70


In [244]:
df_file

Подразделение,date,Подразделение2
0,2023-01-01,47
1,2023-01-02,85
2,2023-01-03,47
3,2023-01-04,46
4,2023-01-05,60
5,2023-01-06,56
6,2023-01-07,47
7,2023-01-08,40
8,2023-01-09,61
9,2023-01-10,70


In [150]:
df1.head()

,Компания,Фио,Подразделение,2023-01-01 00:00:00,2023-01-02 00:00:00,2023-01-03 00:00:00,2023-01-04 00:00:00,2023-01-05 00:00:00,2023-01-06 00:00:00,2023-01-07 00:00:00,...,2023-01-22 00:00:00,2023-01-23 00:00:00,2023-01-24 00:00:00,2023-01-25 00:00:00,2023-01-26 00:00:00,2023-01-27 00:00:00,2023-01-28 00:00:00,2023-01-29 00:00:00,2023-01-30 00:00:00,2023-01-31 00:00:00
0,Подрядчик1,1,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Подрядчик1,2,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9.4,NaN,11.8,NaN,NaN,NaN
2,Подрядчик1,3,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,11.7,NaN,11.7,NaN,NaN
3,Подрядчик1,4,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Подрядчик1,5,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.4,NaN,NaN,NaN,NaN,NaN,NaN


In [227]:
df1c = (df1
    .groupby("Подразделение")
    .count()
    .drop(["Компания", "Фио"], axis=1)
    .T
    .reset_index()
    .rename(columns={"index":"date"})
    )

In [229]:
df_file = pd.DataFrame()

In [232]:
pd.concat([df_file, df1c])

Подразделение,date,Подразделение1
0,2023-01-01,42
1,2023-01-02,84
2,2023-01-03,78
3,2023-01-04,78
4,2023-01-05,61
5,2023-01-06,68
6,2023-01-07,63
7,2023-01-08,61
8,2023-01-09,67
9,2023-01-10,70


In [228]:
df1c

Подразделение,date,Подразделение1
0,2023-01-01,42
1,2023-01-02,84
2,2023-01-03,78
3,2023-01-04,78
4,2023-01-05,61
5,2023-01-06,68
6,2023-01-07,63
7,2023-01-08,61
8,2023-01-09,67
9,2023-01-10,70


In [226]:
df1c.columns

Index(['date', 'Подразделение1'], dtype='object', name='Подразделение')

In [201]:
df1c

0    2023-01-01
1    2023-01-02
2    2023-01-03
3    2023-01-04
4    2023-01-05
5    2023-01-06
6    2023-01-07
7    2023-01-08
8    2023-01-09
9    2023-01-10
10   2023-01-11
11   2023-01-12
12   2023-01-13
13   2023-01-14
14   2023-01-15
15   2023-01-16
16   2023-01-17
17   2023-01-18
18   2023-01-19
19   2023-01-20
20   2023-01-21
21   2023-01-22
22   2023-01-23
23   2023-01-24
24   2023-01-25
25   2023-01-26
26   2023-01-27
27   2023-01-28
28   2023-01-29
29   2023-01-30
30   2023-01-31
Name: date, dtype: datetime64[ns]

In [149]:
(~df1.iloc[:,3:-1].isna()).sum()

2023-01-01 00:00:00    42
2023-01-02 00:00:00    84
2023-01-03 00:00:00    78
2023-01-04 00:00:00    78
2023-01-05 00:00:00    61
2023-01-06 00:00:00    68
2023-01-07 00:00:00    63
2023-01-08 00:00:00    61
2023-01-09 00:00:00    67
2023-01-10 00:00:00    70
2023-01-11 00:00:00    60
2023-01-12 00:00:00    70
2023-01-13 00:00:00    75
2023-01-14 00:00:00    64
2023-01-15 00:00:00    55
2023-01-16 00:00:00    73
2023-01-17 00:00:00    70
2023-01-18 00:00:00    75
2023-01-19 00:00:00    72
2023-01-20 00:00:00    68
2023-01-21 00:00:00    61
2023-01-22 00:00:00    64
2023-01-23 00:00:00    67
2023-01-24 00:00:00    84
2023-01-25 00:00:00    83
2023-01-26 00:00:00    83
2023-01-27 00:00:00    88
2023-01-28 00:00:00    81
2023-01-29 00:00:00    80
2023-01-30 00:00:00    80
dtype: int64

In [53]:
files_dict.keys()

dict_keys(['Подразделение1', 'Подразделение2'])

In [55]:
from dash import Dash, html, dcc, Input, Output
from jupyter_dash import JupyterDash
import pandas as pd
import plotly.express as px

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# app = Dash(__name__, external_stylesheets=external_stylesheets)
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                files_dict.keys,
                'Unit',
                id='unit',
            ),
        ],
        style={}),#'width': '49%', 'display': 'inline-block'}),
        
        html.Div([
            dcc.DatePickerRange(
                id='my-date-picker-range',
                min_date_allowed=date(1995, 8, 5),
                max_date_allowed=date(2017, 8, 5),
                initial_visible_month=date(2017, 8, 5),
                end_date=date(2017, 8, 25)
            ),
            html.Div(id='output-container-date-picker-range')
        ]),

    ], style={}),#'padding': '10px 5px'}),

])

@app.callback(
    Output('output-container-date-picker-range', 'children'),
    Input('my-date-picker-range', 'start_date'),
    Input('my-date-picker-range', 'end_date'))
def update_output(start_date, end_date):
    string_prefix = 'You have selected: '
    if start_date is not None:
        start_date_object = date.fromisoformat(start_date)
        start_date_string = start_date_object.strftime('%B %d, %Y')
        string_prefix = string_prefix + 'Start Date: ' + start_date_string + ' | '
    if end_date is not None:
        end_date_object = date.fromisoformat(end_date)
        end_date_string = end_date_object.strftime('%B %d, %Y')
        string_prefix = string_prefix + 'End Date: ' + end_date_string
    if len(string_prefix) == len('You have selected: '):
        return 'Select a date to see it displayed here'
    else:
        return string_prefix

# @app.callback(
#     Output('crossfilter-indicator-scatter', 'figure'),
#     Input('crossfilter-xaxis-column', 'value'),
#     Input('crossfilter-yaxis-column', 'value'),
#     Input('crossfilter-xaxis-type', 'value'),
#     Input('crossfilter-yaxis-type', 'value'),
#     Input('crossfilter-year--slider', 'value'))
# def update_graph(xaxis_column_name, yaxis_column_name,
#                  xaxis_type, yaxis_type,
#                  year_value):
#     dff = df[df['Year'] == year_value]

#     fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
#             y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
#             hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name']
#             )

#     fig.update_traces(customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

#     fig.update_xaxes(title=xaxis_column_name, type='linear' if xaxis_type == 'Linear' else 'log')

#     fig.update_yaxes(title=yaxis_column_name, type='linear' if yaxis_type == 'Linear' else 'log')

#     fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

#     return fig


# def create_time_series(dff, axis_type, title):

#     fig = px.scatter(dff, x='Year', y='Value')

#     fig.update_traces(mode='lines+markers')

#     fig.update_xaxes(showgrid=False)

#     fig.update_yaxes(type='linear' if axis_type == 'Linear' else 'log')

#     fig.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
#                        xref='paper', yref='paper', showarrow=False, align='left',
#                        text=title)

#     fig.update_layout(height=225, margin={'l': 20, 'b': 30, 'r': 10, 't': 10})

#     return fig


# @app.callback(
#     Output('x-time-series', 'figure'),
#     Input('crossfilter-indicator-scatter', 'hoverData'),
#     Input('crossfilter-xaxis-column', 'value'),
#     Input('crossfilter-xaxis-type', 'value'))
# def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
#     country_name = hoverData['points'][0]['customdata']
#     dff = df[df['Country Name'] == country_name]
#     dff = dff[dff['Indicator Name'] == xaxis_column_name]
#     title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
#     return create_time_series(dff, axis_type, title)


# @app.callback(
#     Output('y-time-series', 'figure'),
#     Input('crossfilter-indicator-scatter', 'hoverData'),
#     Input('crossfilter-yaxis-column', 'value'),
#     Input('crossfilter-yaxis-type', 'value'))
# def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
#     dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
#     dff = dff[dff['Indicator Name'] == yaxis_column_name]
#     return create_time_series(dff, axis_type, yaxis_column_name)


# if __name__ == '__main__':
#     app.run_server(debug=True)
if __name__ == '__main__':
    app.run_server(mode="external")

Dash app running on http://127.0.0.1:8050/


In [115]:
fig = px.bar()

In [ ]:
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')


app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Fertility rate, total (births per woman)',
                id='crossfilter-xaxis-column',
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-xaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Life expectancy at birth, total (years)',
                id='crossfilter-yaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-yaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        df['Year'].min(),
        df['Year'].max(),
        step=None,
        id='crossfilter-year--slider',
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()}
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])


@app.callback(
    Output('crossfilter-indicator-scatter', 'figure'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'),
    Input('crossfilter-yaxis-type', 'value'),
    Input('crossfilter-year--slider', 'value'))
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name']
            )

    fig.update_traces(customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    fig.update_xaxes(title=xaxis_column_name, type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name, type='linear' if yaxis_type == 'Linear' else 'log')

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    return fig


def create_time_series(dff, axis_type, title):

    fig = px.scatter(dff, x='Year', y='Value')

    fig.update_traces(mode='lines+markers')

    fig.update_xaxes(showgrid=False)

    fig.update_yaxes(type='linear' if axis_type == 'Linear' else 'log')

    fig.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                       xref='paper', yref='paper', showarrow=False, align='left',
                       text=title)

    fig.update_layout(height=225, margin={'l': 20, 'b': 30, 'r': 10, 't': 10})

    return fig


@app.callback(
    Output('x-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'))
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@app.callback(
    Output('y-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-yaxis-type', 'value'))
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)


if __name__ == '__main__':
    app.run_server(debug=True)


In [10]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.
from jupyter_dash import JupyterDash
from dash import Dash, html, dcc

app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div(children=[
        html.Label('Dropdown'),
        dcc.Dropdown(['New York City', 'Montréal', 'San Francisco'], 'Montréal'),

        html.Br(),
        html.Label('Multi-Select Dropdown'),
        dcc.Dropdown(['New York City', 'Montréal', 'San Francisco'],
                     ['Montréal', 'San Francisco'],
                     multi=True),

        html.Br(),
        html.Label('Radio Items'),
        dcc.RadioItems(['New York City', 'Montréal', 'San Francisco'], 'Montréal'),
    ], style={'padding': 10, 'flex': 1}),

    html.Div(children=[
        html.Label('Checkboxes'),
        dcc.Checklist(['New York City', 'Montréal', 'San Francisco'],
                      ['Montréal', 'San Francisco']
        ),

        html.Br(),
        html.Label('Text Input'),
        dcc.Input(value='MTL', type='text'),

        html.Br(),
        html.Label('Slider'),
        dcc.Slider(
            min=0,
            max=9,
            marks={i: f'Label {i}' if i == 1 else str(i) for i in range(1, 6)},
            value=5,
        ),
    ], style={'padding': 10, 'flex': 1})
], style={'display': 'flex', 'flex-direction': 'row'})

if __name__ == '__main__':
    app.run_server(mode="external")


Dash app running on http://127.0.0.1:8050/


In [2]:
import pandas as pd


In [10]:
import os

In [7]:
xls1 = pd.ExcelFile("Подразделение1.xlsx")
xls2 = pd.ExcelFile("Подразделение2.xlsx")
df1 = pd.read_excel("Подразделение1.xlsx", 0)
df2 = pd.read_excel("Подразделение1.xlsx", 1)

In [8]:
df2

,Компания,Фио,Подразделение,2023-02-01 00:00:00,2023-02-02 00:00:00,2023-02-03 00:00:00,2023-02-04 00:00:00,2023-02-05 00:00:00,2023-02-06 00:00:00,2023-02-07 00:00:00,...,2023-02-19 00:00:00,2023-02-20 00:00:00,2023-02-21 00:00:00,2023-02-22 00:00:00,2023-02-23 00:00:00,2023-02-24 00:00:00,2023-02-25 00:00:00,2023-02-26 00:00:00,2023-02-27 00:00:00,2023-02-28 00:00:00
0,Подрядчик1,1,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Подрядчик1,2,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Подрядчик1,3,Подразделение1,NaN,NaN,8.6,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Подрядчик1,4,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Подрядчик1,5,Подразделение1,15.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,17.0,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,Подрядчик5,445,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
445,Подрядчик5,446,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,11.0,...,NaN,NaN,11.0,NaN,11.0,11.0,11.0,NaN,NaN,NaN
446,Подрядчик5,447,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.0,11.0,11.0,11.0,11.0,11.0,11.0,NaN,NaN,NaN
447,Подрядчик5,448,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,11.0,...,11.0,11.0,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df1

,Компания,Фио,Подразделение,2023-01-01 00:00:00,2023-01-02 00:00:00,2023-01-03 00:00:00,2023-01-04 00:00:00,2023-01-05 00:00:00,2023-01-06 00:00:00,2023-01-07 00:00:00,...,2023-01-22 00:00:00,2023-01-23 00:00:00,2023-01-24 00:00:00,2023-01-25 00:00:00,2023-01-26 00:00:00,2023-01-27 00:00:00,2023-01-28 00:00:00,2023-01-29 00:00:00,2023-01-30 00:00:00,2023-01-31 00:00:00
0,Подрядчик1,1,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Подрядчик1,2,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9.4,NaN,11.8,NaN,NaN,NaN
2,Подрядчик1,3,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,11.7,NaN,11.7,NaN,NaN
3,Подрядчик1,4,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Подрядчик1,5,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,Подрядчик4 ПРР,359,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,Подрядчик4 ПРР,360,Подразделение1,NaN,11.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
360,Подрядчик4 ПРР,361,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
361,Подрядчик4 ПРР,362,Подразделение1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
